<a href="https://colab.research.google.com/github/maverick98/Coursera/blob/master/CrossWord_Puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from math import inf as infinity
from enum import Enum
import time
import copy
class Direction(Enum):
    ACROSS = 1
    DOWN = 2
class Player(Enum):
    HUMAN=1
    COMPUTER=2
    NEUTRAL=3


class CrosswordPuzzle:

    def __init__(self):
        self.initialize_board()

    def initialize_board(self):
        self.final_board=  [
                            ['#' , 'D' , 'U' , 'C' , 'K' , '#' , 'D'],
                            ['E' , '#' , '#' , 'R' , '#' , '#' , 'O'],
                            ['M' , '#' , '#' , 'O' , '#' , '#' , 'V'],
                            ['U' , '#' , 'S' , 'W' , 'A' , 'N' , 'E'],
                            ['#' , '#' , 'P' , '#' , '#' , '#' , '#'],
                            ['P' , 'E' , 'A' , 'C' , 'O' , 'C' , 'K'],
                            ['#' , '#' , 'R' , '#' , '#' , '#' , '#'],
                            ['P' , 'A' , 'R' , 'R' , 'O' , 'T' , '#'],
                            ['#' , '#' , 'O' , '#' , '#' , '#' , '#'],
                            ['#' , '#' , 'W' , 'R' , 'E' , 'N' , '#']
                          ]
        self.current_board=[
                            ['#' , '_' , '_' , '_' , '_' , '#' , '_'],
                            ['_' , '#' , '#' , '_' , '#' , '#' , '_'],
                            ['_' , '#' , '#' , '_' , '#' , '#' , '_'],
                            ['_' , '#' , '_' , '_' , '_' , '_' , '_'],
                            ['#' , '#' , '_' , '#' , '#' , '#' , '#'],
                            ['_' , '_' , '_' , '_' , '_' , '_' , '_'],
                            ['#' , '#' , '_' , '#' , '#' , '#' , '#'],
                            ['_' , '_' , '_' , '_' , '_' , '_' , '#'],
                            ['#' , '#' , '_' , '#' , '#' , '#' , '#'],
                            ['#' , '#' , '_' , '_' , '_' , '_' , '#']
                          ]
        self.ROWS=len(self.final_board)
        self.COLS=len(self.final_board[0])
        self.across_words_array=self.calculate_across_words()
        self.down_words_array=self.calculate_down_words()
        self.directions=[Direction.ACROSS,Direction.DOWN]

        #The key '2-Across' is a hint to the user.
        #2 can be either row or column. The user has to think accordingly.
        #Across and Down are self explanatory
        #Thus '4-Down': 'SPARROW' means The user has to put SPARROW on either 4th row or 4th column
        #Vertically down. Please note, another hint could have been '3-DOWN'. The coordinate of 'S' of SPARROW is
        #(4,3)
        #However the the computer knows the final position of the board. Thus we (the computer) do not need such hint
        # to play the game.
        self.words = {
                        'SPARROW':'4-Down',
                        'DUCK':'2-Across',
                        'WREN':'3-Across',
                        'SWAN':'4-Across',
                        'CROW':'4-Down',
                        'DOVE':'7-Down',
                        'PEACOCK':'6-Across',
                        'PARROT':'8-Across',
                        'EMU':'2-Down'
                      }
        self.words_actual_coordinates=self.calculate_actual_coordinates_of_words();
        self.player_turn=Player.HUMAN
        self.human_score=0
        self.computer_score=0
        self.words_stack=self.sort_words()
    def sort_words(self):
        words_stack=[]
        for word,_ in self.words.items():
            words_stack.append(word)
        words_stack.sort(key = lambda x : len(x))
        return words_stack
    def calculate_across_words(self):
        result=[]
        for i in range(0, self.ROWS):
            across_words="".join(self.final_board[i])
            result.append(across_words)
        return result
    def calculate_down_words(self):
        result=[]
        for j in range(0, self.COLS):
            down_words=[]
            for i in range(0, self.ROWS):
                down_words.append(self.final_board[i][j])
            result.append("".join(down_words))
        return result
    def get_actual_coordinate(self,word):
        return self.words_actual_coordinates[word]
    def calculate_actual_coordinate(self,word):
        tmp = [across_words.find(word) for across_words in self.across_words_array]
        row=-1
        col=-1
        direction=None
        if any(num >=0 for num in tmp):
            col=sum([ idx for idx in tmp if idx >=0])
            row=tmp.index(col)
            direction=Direction.ACROSS
        else:
            tmp = [down_words.find(word) for down_words in self.down_words_array]
            if any(num >=0 for num in tmp):
               row=sum([ idx for idx in tmp if idx >=0])
               col=tmp.index(row)
               direction=Direction.DOWN
        return row,col,direction
    def calculate_actual_coordinates_of_words(self):
        result={}
        for word,_ in self.words.items():
            row,col,direction=self.calculate_actual_coordinate(word)
            result[word]=(row,col,direction)
        return result
    def show_actual_coordinates_of_words(self):
        for word,(row,col,direction) in self.words_actual_coordinates.items():
            print('{} = ({},{},{})'.format(word,row,col,direction))
    def draw_board(self):
        for i in range(0, self.ROWS):
            for j in range(0, self.COLS):
                print('{}|'.format(self.current_board[i][j]), end=" ")
            print()
        print()

    def is_valid_move(self, word, row, col, direction):
        if direction == Direction.ACROSS:
            if len(word) > self.COLS - col:
                return False
            for i, word_char in enumerate(word):
                if self.current_board[row][col + i] != '_' and self.current_board[row][col + i] != word_char:
                    return False
        elif direction == Direction.DOWN:
            if len(word) > self.ROWS - row:
                return False
            for i, word_char in enumerate(word):
                if self.current_board[row + i][col] != '_' and self.current_board[row + i][col] != word_char:
                    return False
        return True
    def test_is_valid_move(self):
        print(self.is_valid_move('DUCK',0,1,Direction.ACROSS))
        print(self.is_valid_move('DUCK',0,1,Direction.DOWN) == False)
        print(self.is_valid_move('DOVE',0,6,Direction.DOWN))
        print(self.is_valid_move('DOVE',0,6,Direction.ACROSS)==False)
        print(self.is_valid_move('PEACOCK',5,0,Direction.ACROSS))
        print(self.is_valid_move('PEACOCK',5,0,Direction.DOWN) == False)
        print(self.is_valid_move('PARROT',7,0,Direction.ACROSS))
        print(self.is_valid_move('PARROT',7,0,Direction.DOWN) == False)
        print(self.is_valid_move('WREN',9,2,Direction.ACROSS))
        print(self.is_valid_move('WREN',9,2,Direction.DOWN) == False)
        print(self.is_valid_move('EMU',1,0,Direction.DOWN))
        print(self.is_valid_move('EMU',1,0,Direction.ACROSS) == False)
        print(self.is_valid_move('CROW',0,3,Direction.DOWN))
        print(self.is_valid_move('CROW',0,3,Direction.ACROSS) == False)
        print(self.is_valid_move('SWAN',3,2,Direction.ACROSS))
        print(self.is_valid_move('SWAN',3,2,Direction.DOWN) == True)
        #SWAN can be put vertically down. That's when we will punish the user with -1 marks
        #We, the computer, will put it across at (3,2)
    def place_word(self,board,should_clone, word, row, col, direction):
        new_board=board
        if should_clone == True:
            new_board = copy.deepcopy(board)

        if direction == Direction.ACROSS:
            for i, word_char in enumerate(word):
                new_board[row][col + i] = word_char
        elif direction == Direction.DOWN:
            for i, word_char in enumerate(word):
                new_board[row + i][col] = word_char
        return new_board
    def is_endofgame(self):
        game_finished=False
        empty_space_count=sum([arr.count('_') for arr in self.current_board])
        if empty_space_count == 0 or len(self.words_stack) == 0:
            game_finished=True
        if game_finished == True:
           game_winner = Player.NEUTRAL
           if self.human_score > self.computer_score:
                game_winner = Player.HUMAN
           elif self.human_score < self.computer_score:
                game_winner = Player.COMPUTER
           else:
                game_winner=Player.NEUTRAL
           return game_winner
        return None

    def max_alpha_beta(self,board,word, alpha, beta):
        maxv = -infinity
        px = None
        py = None
        p_direction = None

        result = self.is_endofgame()

        if result == Player.HUMAN:
            return (-1, 0, 0,None)
        elif result == Player.COMPUTER:
            return (1, 0, 0,None)
        elif result == Player.NEUTRAL:
            return (0, 0, 0,None)

        for i in range(0, self.ROWS):
            for j in range(0, self.COLS):
                for direction in self.directions:
                    if self.is_valid_move(word, i, j, direction):
                        new_board=self.place_word(board,True,word, i, j, direction)
                        next_word=self.words_stack.pop()
                        (m, _, _,_) = self.min_alpha_beta(new_board,next_word,alpha, beta)
                        self.words_stack.append(next_word)
                        if m > maxv:
                            maxv = m
                            px = i
                            py = j
                            p_direction=direction
                        #empty_word=''.join(['_' for _ in word])
                        #self.place_word(empty_word, i, j, direction)

                        # Next two ifs in Max and Min are the only difference between regular algorithm and minimax
                        if maxv >= beta:
                            return (maxv, px, py,p_direction)

                        if maxv > alpha:
                            alpha = maxv

        return (maxv, px, py , p_direction)
    def min_alpha_beta(self,board,word, alpha, beta):

        minv = infinity

        qx = None
        qy = None
        q_direction = None

        result = self.is_endofgame()

        if result == Player.HUMAN:
            return (-1, 0, 0,None)
        elif result == Player.COMPUTER:
            return (1, 0, 0,None)
        elif result == Player.NEUTRAL:
            return (0, 0, 0,None)

        for i in range(0, self.ROWS):
            for j in range(0, self.COLS):
                for direction in self.directions:
                    if self.is_valid_move(word, i, j, direction):
                        new_board=self.place_word(board,True,word, i, j, direction)
                        next_word=self.words_stack.pop()
                        (m, _, _,_) = self.max_alpha_beta(new_board,next_word,alpha, beta)
                        self.words_stack.append(next_word)
                        if m < minv:
                            minv = m
                            qx = i
                            qy = j
                            q_direction = direction
                        #empty_word=''.join(['_' for _ in word])
                        #self.place_word(empty_word, i, j, direction)

                        if minv <= alpha:
                            return (minv, qx, qy,q_direction)

                        if minv < beta:
                            beta = minv

        return (minv, qx, qy,q_direction)
    def calculate_score(self,word,row,col,direction):
        score=None
        valid_position=False
        actual_row,actual_col,actual_direction=self.get_actual_coordinate(word)
        if actual_row == row and actual_col== col and actual_direction==direction:
            valid_position=True
            score=len(word)
        else:
            valid_position=False
            score =-1
        return valid_position,actual_row,actual_col,actual_direction,score
    def play_alpha_beta(self):
        board = self.current_board
        while len(self.words_stack) !=0:
            word = self.words_stack.pop()


            self.draw_board()
            self.result = self.is_endofgame()

            if self.result != None:
                if self.result == Player.HUMAN:
                    print('The winner is Human!')
                elif self.result == Player.COMPUTER:
                    print('The winner is Computer!')
                elif self.result == Player.NEUTRAL:
                    print("It's a tie!")


                self.initialize_board()
                return

            if self.player_turn ==  Player.HUMAN:
                hint=self.words[word]
                #print('Current Word is {} , hint {}'.format(word,hint))
                print('Current Word is {}'.format(word))

                while True:
                    row = int(input('Insert the Row coordinate: '))
                    col = int(input('Insert the Col coordinate: '))
                    dir_int = int(input('Insert the Direction(1=Across,2=Down): '))
                    word_direction=Direction.ACROSS
                    if dir_int == 1:
                        word_direction=Direction.ACROSS
                    else:
                        word_direction=Direction.DOWN



                    if self.is_valid_move(word,row,col,word_direction):
                        valid_position,actual_row,actual_col,actual_direction,score=self.calculate_score(word,row,col,word_direction)
                        self.human_score +=score
                        if valid_position == True:
                            self.place_word(board,False,word,row,col,word_direction)
                        else:
                            time.sleep(5)
                            print('Your move is wrong! Let me place it at right position for you')
                            self.computer_score +=score
                            self.place_word(board,False,word,actual_row,actual_col,actual_direction)



                        self.player_turn =  Player.COMPUTER
                        break
                    else:
                        print('The move is not valid! Try again.')

            else:
                (_, row,col,word_direction) = self.max_alpha_beta(board,word,-infinity, infinity)
                self.place_word(board,False,word,row,col,word_direction)
                self.player_turn =  Player.HUMAN


def main():
    game = CrosswordPuzzle()
    #game.draw_board()
    #game.test_is_valid_move()
    #game.place_word('SPARROW',3,2,Direction.DOWN)
    #game.draw_board()
    #print(game.is_endofgame())

    game.show_actual_coordinates_of_words()
    #print(game.get_actual_coordinate('SPARROW'))
    #game.play_alpha_beta()


if __name__ == "__main__":
    main()




SPARROW = (3,2,Direction.DOWN)
DUCK = (0,1,Direction.ACROSS)
WREN = (9,2,Direction.ACROSS)
SWAN = (3,2,Direction.ACROSS)
CROW = (0,3,Direction.DOWN)
DOVE = (0,6,Direction.DOWN)
PEACOCK = (5,0,Direction.ACROSS)
PARROT = (7,0,Direction.ACROSS)
EMU = (1,0,Direction.DOWN)
